In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Đọc file RFM

import pandas as pd

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv("/content/drive/MyDrive/LDS0_K292_ONLINE_HaThiThieuDao/Project03/RFM.csv")

# Nhóm dữ liệu theo CustomerID
data = data.groupby('CustomerID').agg({'Recency': 'sum',
                                    'Frequency': 'sum',
                                    'Monetary': 'sum'}).reset_index()
data.loc[:10]

#Phân cụm khách hàng thủ công
# Chia R F M ra 5 nhóm theo quintile, gán điểm cho mỗi nhóm, tạo cột RFMscore
import pandas as pd

# Chia R, F, M ra 5 nhóm theo quintile và gán điểm cho mỗi nhóm
r_score = pd.qcut(data['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')
f_score = pd.qcut(data['Frequency'], 5, labels=[1, 2, 3, 4, 5])
m_score = pd.qcut(data['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Thêm các điểm vào DataFrame
data['R'] = r_score
data['F'] = f_score
data['M'] = m_score

# Tính RFMscore
data['RFMscore'] = data['R'].astype(int) + data['F'].astype(int) + data['M'].astype(int)
data.head()


# Hiển thị tất cả các cột khi in ra DataFrame
pd.set_option('display.max_columns', None)
# Mô tả RFMscore
print(data['RFMscore'].describe().round(0))

# Phân 5 nhóm khách hàng (dựa vào tính thường xuyên và giá trị của giao dịch)
# 1. Khách hàng VIP	13-15	Mua gần đây, thường xuyên, giá trị cao
# 2. Khách hàng thân thiết	11-13	Mua gần đây, thường xuyên, giá trị trung bình
# 3. Khách hàng tiềm năng	9-11	Mua cách đây vừa phải, thường xuyên, giá trị trung bình
# 4. Khách hàng cần quan tâm 7-9	Mua cách đây lâu, không thường xuyên, giá trị trung bình
# 5. Khách hàng bàng quan	5-7	Mua cách đây lâu, không thường xuyên
def customer_segment(rfm_score):
  if rfm_score >= 13:
    return "VIP"
  elif 11 <= rfm_score < 13:
    return "Thân thiết"
  elif 9 <= rfm_score < 11:
    return "Tiềm năng"
  elif 7 <= rfm_score < 9:
    return "Quan tâm"
  else:
    return "Bàng quan"

data['Customer_Segment'] = data['RFMscore'].apply(customer_segment)

# In 5 dòng đầu của
data.head()

import pandas as pd
import plotly.express as px

# Thêm cột trung bình Recency, Frequency và Monetary cho mỗi nhóm khách hàng
data_grouped = data.groupby('Customer_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).reset_index()

# Thêm cột mới vào DataFrame
data_grouped = data_grouped.assign(
    Recency_mean=data_grouped['Recency'],
    Frequency_mean=data_grouped['Frequency'],
    Monetary_mean=data_grouped['Monetary']
)

# In ra DataFrame kết quả
print(data_grouped)


# Tạo biểu đồ treemap
fig = px.treemap(data_grouped,
                 path=['Customer_Segment'],
                 values='Recency',
                 color='Customer_Segment',
                 color_discrete_map={
                     'VIP': '#129cab',
                     'Thân thiết': '#17becf',
                     'Tiềm năng': '#92c5de',
                     'Quan tâm': '#f4d7a4',
                     'Bàng quan': '#d6a585'
                 },
                 title='Phân khúc khách hàng và Recency, Frequency, Monetary trung bình',
                 labels={'Customer_Segment': 'Phân khúc',
                         'Recency': 'Recency trung bình'},
                 hover_data=['Frequency', 'Monetary'])  # Thêm thông tin về Frequency và Monetary

fig.update_traces(textinfo='label+value+percent parent')

fig.update_layout(
    height=600,
    width=800,
    margin=dict(t=50, r=10, l=10, b=10)
)

fig.show()
# In các nhóm với các giá trị trung bình của 'Recency', 'Frequency', 'Monetary', số lượng khách hàng mỗi nhóm và tỷ lệ % so với tổng số khách hàng

import pandas as pd

# Nhóm dữ liệu theo Customer_Segment
grouped_data = data.groupby('Customer_Segment')

# Tính giá trị trung bình của 'Recency', 'Frequency', 'Monetary' cho mỗi nhóm
rfm_means = grouped_data[['Recency', 'Frequency', 'Monetary']].mean().reset_index()

# Đếm số lượng khách hàng trong mỗi nhóm
customer_counts = grouped_data['CustomerID'].count().reset_index(name='Count')

# Tính tỷ lệ phần trăm khách hàng trong mỗi nhóm
customer_percentages = (customer_counts['Count'] / customer_counts['Count'].sum() * 100).round(2).reset_index(name='% of Customers')

# Kết hợp các DataFrame lại với nhau
rfm_summary = pd.concat([rfm_means, customer_counts, customer_percentages], axis=1)

# In kết quả
print(rfm_summary.to_string())

customer_id = input("Nhập mã khách hàng (CustomerID): ")

# Kiểm tra xem CustomerID được cung cấp có tồn tại trong DataFrame không
customer_data = data[data['CustomerID'] == int(customer_id)]  # Giả sử CustomerID được lưu dưới dạng số nguyên

if not customer_data.empty:
    customer_segment = customer_data['Customer_Segment'].values[0]
    print(f"Mã khách hàng {customer_id} thuộc nhóm {customer_segment}.")
else:
    print(f"Mã khách hàng {customer_id} không tồn tại trong dữ liệu.")

# prompt: lưu mô hình pickle

import pickle

# Train the model
# ...

# Save the model to a file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# model.pkl đang nằm ở thư mục nào đưa về đây /content/drive/MyDrive/LDS0_K292_ONLINE_HaThiThieuDao/Project03

!cp model.pkl "/content/drive/MyDrive/LDS0_K292_ONLINE_HaThiThieuDao/Project03/model.pkl"

# goi mô hình cung cấp CustomerID và mô hình dự đoán nhóm

# Load the model from the pickle file
with open('/content/drive/MyDrive/LDS0_K292_ONLINE_HaThiThieuDao/Project03/model.pkl', 'rb') as file:
    model = pickle.load(file)

# Get the customer ID from the user
customer_id = int(input("Nhập mã khách hàng (CustomerID): "))

# Check if the customer ID exists in the model
if customer_id not in model['CustomerID'].values:
    print(f"Mã khách hàng {customer_id} không tồn tại trong dữ liệu.")
else:
    # Get the customer segment for the given customer ID
    customer_segment = model[model['CustomerID'] == customer_id]['Customer_Segment'].values[0]
    print(f"Mã khách hàng {customer_id} thuộc nhóm {customer_segment}.")

# describe data

data.describe()


In [ ]:
# Hãy viết recency (trong khoảng: min-max), tương tự cho frequency và monetary

print(f"Recency (min-max): {data['Recency'].min()}-{data['Recency'].max()}")
print(f"Frequency (min-max): {data['Frequency'].min()}-{data['Frequency'].max()}")
print(f"Monetary (min-max): {data['Monetary'].min()}-{data['Monetary'].max()}")
